# 1. 통계학 및 모델 구축과 검증에 관한 개념들

## 1-1. 기술통계
- 평균, 중앙, 최빈값
    - mode 함수를 구현하기 위해 SciPy의 Stats 패키지 활용

In [6]:
import numpy as np
from scipy import stats

data = np.array([4, 5, 1, 2, 7, 2, 6, 9, 3])

## 평균 계산
dt_mean = np.mean(data) ; print("Mean :", round(dt_mean, 2))
print('-'*30)

## 중앙값 계산
dt_median = np.median(data) ; print("Median :", dt_median)
print('-'*30)

## 최빈값 계산
dt_mode = stats.mode(data) ; print("Mode :", dt_mode[0][0])

Mean : 4.33
------------------------------
Median : 4.0
------------------------------
Mode : 2


## 1-2. 산포
- 분산, 표준편차, 범위, 분위틀, IQR

In [5]:
from statistics import variance, stdev

game_points = np.array([35, 36, 43, 59, 63, 79, 35, 41, 64, 43, 93, 60, 77, 24, 82])

## 분산 계산
dt_var = variance(game_points)
print("Sample variance :", round(dt_var, 2))
print('-'*30)

## 표준편차 계산
dt_std = stdev(game_points)
print("Sample stdev :", round(dt_std, 2))
print('-'*30)

## 범위 계산
dt_rng = np.max(game_points, axis=0) - np.mean(game_points, axis=0) ; print("Range :", dt_rng)
print('-'*30)

## 백분위 계산
print("Quantiles :")
for val in [20, 80, 100]:
    dt_qntls = np.percentile(game_points, val)
    print(str(val)+"%", dt_qntls)
print('-'*30)

## IQR 계산
q75, q25 = np.percentile(game_points, [75, 25])
print("Inter quartile range :", q75-q25)

Sample variance : 429
------------------------------
Sample stdev: 20.71
------------------------------
Range : 37.4
------------------------------
Quantiles: 
20% 35.800000000000004
80% 77.4
100% 93.0
------------------------------
Inter quartile range: 32.0


## 1-3. T-분포
- t = (x_bar - H0) / (S/sqrt(n))
- 평균 : 990, 귀무가설 : >= 1000, 표준편차 : 12.5, 표본수 : 30
    - t = (990 - 1000) / (12.5 / sqrt(30))

In [12]:
from scipy import stats
xbar = 990; mu0 = 1000; s = 12.5; n=30

## 검정 통계값
t_sample = (xbar - mu0) / (s/np.sqrt(float(n)))
print("Test Statistic :", round(t_sample, 2))

Test Statistic:  -4.38


### Example 1
- 입학시험 점수는 정규분포를 따른다고 가정하자. 평균 점수는 52점이고, 표준편차는 16.3인 경우, 몇 %의 학생들이 67점 이상을 받을까?

In [15]:
from scipy import stats
xbar = 67; mu0 = 52; s = 16.3

## Z-score 계산
z = (67 - 52) / 16.3
print("Z-score: ", z)
print('-'*30)

## 곡선하 확률 면적 계산
p_val = 1 - stats.norm.cdf(z)
print("Prob. to score more than 67 is", round(p_val*100, 2), "%")

Z-score:  0.920245398773006
------------------------------
Prob. to score more than 67 is 17.87 %


## 1-4. 카이-제곱 (Chi-square)
- 범주형 데이터의 통계 분석에 가장 보편적으로 사용되는 검정 중 하나
- 두 변수 사이에 통계적 상관성이 존재하는지를 판단

In [18]:
import pandas as pd
from scipy import stats

survey = pd.read_csv('./Statistics_for_MachineLearning_Code/Chapter01/Data/survey.csv')

print(survey.shape)
survey.head()

(237, 12)


,Sex,Wr.Hnd,NW.Hnd,W.Hnd,Fold,Pulse,Clap,Exer,Smoke,Height,M.I,Age
0,Female,18.5,18.0,Right,R on L,92.0,Left,Some,Never,173.0,Metric,18.250
1,Male,19.5,20.5,Left,R on L,104.0,Left,None,Regul,177.8,Imperial,17.583
2,Male,18.0,13.3,Right,L on R,87.0,Neither,None,Occas,NaN,NaN,16.917
3,Male,18.8,18.9,Right,R on L,NaN,Neither,None,Never,160.0,Metric,20.333
4,Male,20.0,20.0,Right,Neither,35.0,Right,Some,Never,165.0,Metric,23.667


In [20]:
print(survey['Exer'].unique())
print('-'*30)
print(survey['Smoke'].unique().tolist())

['Some' 'None' 'Freq']
------------------------------
['Never', 'Regul', 'Occas', 'Heavy', nan]


In [21]:
## 2개의 변수를 표(행렬)로 만듦

survey_tab = pd.crosstab(survey.Smoke, survey.Exer, margins=True)
survey_tab

Exer,Freq,None,Some,All
Smoke,,,,
Heavy,7,1,3,11
Never,87,18,84,189
Occas,12,3,4,19
Regul,9,1,7,17
All,115,23,98,236


In [25]:
## 분석을 위해 관측값 표 생성
observed = survey_tab.iloc[0:4, 0:3]
observed

Exer,Freq,None,Some
Smoke,,,
Heavy,7,1,3
Never,87,18,84
Occas,12,3,4
Regul,9,1,7


In [26]:
contg = stats.chi2_contingency(observed=observed)
contg

(5.488545890584232,
 0.48284216946545633,
 6,
 array([[ 5.36016949,  1.0720339 ,  4.56779661],
        [92.09745763, 18.41949153, 78.48305085],
        [ 9.25847458,  1.85169492,  7.88983051],
        [ 8.28389831,  1.65677966,  7.05932203]]))

In [40]:
p_value = round(contg[1], 3)
print("P-value :", p_value)
print('-'*30)

if p_value >= 0.05:
    print("두 변수 사이에는 통계적인 상관성이 없다. 통계적으로 무의미하다.")
    print("즉, 흡연 습관과 운동 행동에는 상관관계가 없다는 것을 알 수 있다.")
else:
    print("두 변수에는 통계적인 상관성이 있다. 통계적으로 유의미하다.")
    print("흡연 습관과 운동행동에는 상관관계가 있다는 것을 알 수 있다.")

P-value : 0.483
------------------------------
두 변수 사이에는 통계적인 상관성이 없다. 통계적으로 무의미하다.
즉, 흡연 습관과 운동 행동에는 상관관계가 없다는 것을 알 수 있다.


## 1-5. 아노바(Anova) 분산 검정
- 둘 이상 모집단의 평균이 서로 동일한지 테스트
    - 귀무가설 : 모든 모집단의 평균은 같다.
    - 대립가설 : 적어도 하나의 모집단은 평균이 다르다.

### Example 2
- 어떤 비료회사는 오랜 연구 끝에 모든 작물에 사용 가능한 세 가지 범용 비료를 개발해냈다. 세 가지 비료가 모두 비슷한 수확량을 거두는지 알아보기 위해 여섯 가지 곡식을 임의로 선택해 연구해보기로 했다. 임의로 구역을 설정한 후 여섯 가지 작물에 관해 세 가지 비료의 효과를 각각 테스트한다. 다음 표는 단위 제곱미터당 생산량을 그램으로 표시했다. 유의수준은 0.05로 세 비료의 평균 수확량이 동일한지 테스트해보라.

In [34]:
## Import Library
import pandas as pd
from scipy import stats

## Data Load
fetilizers = pd.read_csv("./Statistics_for_MachineLearning_Code/Chapter01/Data/fetilizers.csv")

print(fetilizers.shape)
fetilizers

(6, 3)


,fertilizer1,fertilizer2,fertilizer3
0,62,54,48
1,62,56,62
2,90,58,92
3,42,36,96
4,84,72,92
5,64,34,80


In [35]:
## stats 패키지를 이용해 Anova 일원 분산 분석을 계산
one_way_anova = stats.f_oneway(fetilizers["fertilizer1"],
                               fetilizers["fertilizer2"],
                               fetilizers["fertilizer3"])
one_way_anova

F_onewayResult(statistic=3.6634935025687523, pvalue=0.05063590143901569)

In [41]:
print("Statistic :", round(one_way_anova[0], 2), "P-value :", round(one_way_anova[1], 3))
print('-'*30)

if round(one_way_anova[1], 3) >= 0.05:
    print("귀무가설 기각, 변수 사이에는 통계적인 상관성이 없다(무의미).")
    print("세 비료의 평균 수확량이 동일하지 않다고 볼 수 있다.")
    print("비료에 따라 농작물에 끼치는 생산량이 다르다.")
else:
    print("귀무가설 채택, 변수 사이에는 통계적인 상관성이 있다(유의미).")
    print("세 비료의 평균 수확량이 동일하다고 볼 수 있다.")
    print("비료에 따라 농작물에 끼치는 생산량이 같다고 볼 수 있다.")
    

Statistic : 3.66 P-value : 0.051
------------------------------
귀무가설 기각, 변수 사이에는 통계적인 상관성이 없다(무의미).
세 비료의 평균 수확량이 동일하지 않다고 볼 수 있다.
비료에 따라 농작물에 끼치는 생산량이 다르다.


### Example 2 결론
- P 값이 0.05보다 작다. 따라서 농작물 간의 평균 생산성이 동일하다는 귀무가설을 기각한다. 비료에 따라 농작물에 끼치는 생산량이 상당히 다르다는 것을 알 수 있다.

## 1-6. Confusion matrix(혼동행렬)
- 예측값이 실제 관측값을 얼마나 정확히 예측했는지 보여주는 행렬이다.


## 1-7. Performance window(관측과 성능 창)
- 과거 24개월치 데이터로부터 추출한 독립 변수와 향후 12개월 예측을 수행할 종속 변수를 사용해 디폴트 확률 모델(행동 점수, behavioral scorecard)를 개발해야 한다.
- X와 Y변수에 관한 데이터가 준비되면 훈련 데이터와 테스트 데이터를 각각 7:3으로 임의로 나눈다.
- 이 방법은 훈련과 테스트 데이터를 모두 동일한 기간에서 가져오므로 인타임 검증(in-time validation)이라 불린다.

## 1-8. 인타임(in-time)과 아웃타임(out-of-time) 검증
- __인타임 검증__은 훈련 데이터 세트와 테스트 데이터 세트가 모두 __동일한 시간 간격__에서 추출됐음을 의미하고,
- __아웃타임 검증__은 두 데이터 세트가 각기 __다른 시간 간격_에서 추출됐다는 것을 의미한다.
- __대개 모델은 아웃타임 검증에서 더 좋지 않은 결과를 나타내는데, 이는 훈련 세트와 데이터 세트의 특성이 다를 수 있으므로 당연한 결과기도 하다.__

## 1-9. R-제곱(결정계수 coefficient of determination)
- R-제곱은 모델에 의해 설명된 반응 변수 분산의 퍼센티지 비율을 측정하는 것이다(즉, 모든 점에 관해 모델과의 거리를 제곱해서 이전보다 줄어든 점의 비율을 측정한다).
- __이 모델은 단순히 평균만 이용했을 때보다 오류를 얼마나 더 최소화하는지 측정한다.__
- 극단적인 경우, R-제곱이 0보다 작아질 때가 있는데, 이때는 단순 평균만 사용해 모든 예측값을 계산한 것보다도 오히려 더 좋지 않은 결과를 나타낸다는 의미다.
    
## 1-10. 수정된 R-제곱 (Adjusted R-squared)
- 수정 R-제곱 통계량은 기본적으로 R 제곱과 동일하지만, 모델에 새로운 변수가 추가되면 그 변수가 강한 상관관계를 갖지 않는 한 일정 점수를 감점한다.
- 수정된 R-제곱은 __선형회귀의 품질을 평가__하는 __핵심 척도__다. 선형 회귀 모델의 수정 R-제곱값이 __0.7 이상이면__ 구현해도 충분한 정도의 __좋은 모델__로 여겨진다.
- 요약 : __독립 변수의 수가 증가하면 유의하든 유의하지 않든 결정계수가 커지는 R-제곱의 단점을 보완__

## 1-11. 최대 우도 추정(Maximum likehood estimate, MLE)
- 이 기법은 주어진 관측값이 생성되기 위한 우도(likehood)를 최대화하는 매개변숫값을 찾음으로써 __통계모델(로지스틱 회귀)의 매개변숫값을 계산하는 것__이다.
- 이 주제는 로지스틱 회귀, 랜덤포레스트에서 자세히 다룸

## 1-12. 아카이케 정보 기준(Akaike information criteria, AIC)
- AIC는 __로지스틱 회귀__에서 사용되는데 선형 회귀의 수정 R-제곱과 유사한 원칙을 갖고 있다. AIC는 __주어진 데이터 집합의 상대적인 품질을 측정__한다.
- AIC의 기본 아이디어는 __모델에 추가된 변수가 강한 예측 능력값을 갖고 있지 않으면 목적 함수(objective function)를 감점__하는 것이다.
- 이를 통해 로지스틱 회귀에 일종의 __정규화(regularization)__ 과정을 수행한다.

## 1-13. 엔트로피 (Entropy)
- 엔트로피는 정보 이론(information theory)에서 가져온 개념으로 모델의 __불순도(impurity)를 측정__한다.
- 표본이 완전 균질이면 엔트로피는 0이 되고, 표본이 동등하게 분리돼 있다면 1이 된다.
- 주어진 데이터를 그리디 모드(greedy mode)로 분류할 때 의사결정 트리에서 가장 비균질한 예측값이 루트 노드에 가장 가깝게 위치한다.
- 이 주제는 추후 트리기반 머신러닝 모델에서 자세히 다룸
- 엔트로피는 중앙에서 최대값 1이 되고, 양 끝단에서 최솟값 0을 가진다. 낮은 값의 엔트로피는 부류를 더 잘 분리할 수 있으므로 더 바람직하다.

## 1-14. 정보이득 (Information Gain)
- 정보 이득이란 주어진 속성으로 관측값을 분할함에 따라 얻을 수 있는 예상 엔트로피 감소를 의미한다. 기본적인 아이디어는 혼합 부류에서 출발해서 각 노드의 관측값이 순수 부류가 될 때까지 반복적으로 분할해 나가는 것이다. 각 단계마다 그리디 모드로 최대 정보 이득값을 가진 변수를 선택한다.
    - 정보이득 = 부모 노드의 엔트로피 - sum(가중값% * 자식 노드의 엔트로피)
    - 가중값 % = 특정 자식 노드의 관측값 수 / 모든 자식 노드의 관측값 수

## 1-15. 지니(Gini)
- 지니 불순도(gini impurity)는 오분류를 측정하는 도구로, 다부류 분류기에 적용된다. 지니는 엔트로피와 거의 동일하지만 훨씬 더 빨리 계산할 수 있다. 

# 2. 편향 분산의 트레이드 오프
- __모든 모델__은 __백색잡음(white noise)__과 함께 __편향(bias) 및 분산(variance) 오류__를 갖고 있다.
- 편향과 분산은 서로 __역상관관계__를 갖고 있어서 하나를 줄이면 다른 하나는 증가한다. 이 둘의 적절한 조화를 찾아내는 것은 한 마디로 예술의 경지다.
- 이상적인 모델은 낮은 편향과 동시에 낮은 분산을 가져야 한다.
- __편향 오류__는 기본 __학습 알고리즘의 잘못된 가정에서 비롯__된다. 높은 편향값은 알고리즘이 데이터의 특징(feature)과 목표 결과물과의 적절한 관계를 놓치게 만드는 __과소 적합(underfitting)문제를 초래__한다.
- 반면 __분산 오류__는 모델 적합화의 변화에 관한 민감도에서 기인한다. 훈련데이터의 미미한 변화에도 높은 분산의 경우는 __과적합(overfitting)문제__를 일으킬 수 있다.

- __높은 편향 모델의 예__는 __'로지스틱'__ 또는 __'선형회귀'__가 있다. 모델 적합화가 단순히 직선만으로 이뤄지므로 기본 데이터를 잘 근사하지 못하는 한계로 인해 높은 오류 성분을 가질 수 있다.
- __높은 분산 모델의 예__는 __의사결정 트리__다. 모델이 너무 구불구불한 곡선으로 적합화된 경우에는 훈련데이터의 미미한 변화에도 적합화 상태가 크게 변할 수 있다.

- __현 시점의 최첨단 모델__은 의사결정 트리 같이 분산이 높은 모델을 활용하되, 그 위에 __앙상블(ensemble)기술을 접목__해 높은 분산으로부터 발생하는 오류를 최소화하고 동시에 편향에서 야기되는 오류가 너무 커지지 않게 한다. 이런 범주의 모델 중 가장 좋은 예는 __랜덤 포레스트(random forest)__다. 랜덤 포레스트 기법에서는 __여러 의사결정 트리가 서로 독립적으로 성장하다가 최적의 적합화를 찾기 위해 앙상블 과정을 거친다.__

# 3. 훈련 데이터와 테스트 데이터
- 통계 모델링은 대개 데이터를 70:30이나 80:20으로 나눠 각각 훈련 데이터 집합(training set)과 테스트 데이터 집합(test set)으로 사용한다.
- training set은 모델을 구축하는 데 사용하고, 모델의 성능은 test set을 사용해 확인한다.
- Data를 training set과 test set으로 나눌 때, __중요하게 고려해야 할 점__은 매__번 훈련과 테스트 데이터에 동일한 관측값이 생성되도록 하기 위해__ __난수의 시드를 고정시켜야 한다__는 것이다.
- __결과 재현을 위해서는 동일한 관측값의 반복이 대단히 중요하다.__


In [44]:
##### 훈련 & 테스트 분할

## Import Library
import pandas as pd
from sklearn.model_selection import train_test_split

## Data Load
original_data = pd.read_csv('./Statistics_for_MachineLearning_Code/Chapter01/Data/mtcars.csv')

## Data Check
print(original_data.shape)
original_data.head()

(32, 12)


,Unnamed: 0,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [51]:
origianl_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 12 columns):
Unnamed: 0    32 non-null object
mpg           32 non-null float64
cyl           32 non-null int64
disp          32 non-null float64
hp            32 non-null int64
drat          32 non-null float64
wt            32 non-null float64
qsec          32 non-null float64
vs            32 non-null int64
am            32 non-null int64
gear          32 non-null int64
carb          32 non-null int64
dtypes: float64(5), int64(6), object(1)
memory usage: 3.1+ KB


In [55]:
## 데이터 분할
# train_size 0.7 == 70%, random_state == 유사 난수 발생기의 시드를 고정
# 동일한 관측값을 생성해 동일 결과를 재현하기 위해서 시드를 고정
train_data, test_data = train_test_split(original_data, train_size = 0.7, random_state=42)

print(train_data.shape)
print(test_data.shape)

(22, 12)
(10, 12)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# 4. 모델 구축과 검증을 위한 머신 러닝 용어
- __통계 모델링__은 __두 독립 변수를 사용한 선형 회귀를 통해 오차가 최소화 되는 최적 평면을 적합화__하는 반면
- __머신 러닝__은 __독립변수들을 오차항의 제곱으로 변환__(제곱으로 변환하므로 볼록 곡선이 보장되고 따라서 수렴이 빨라지는 동시에 전역 최적임이 동시에 보장된다)하고
- 독립 변수에 관해서가 아니라 __계수값(coefficient values)에 관해 최적화를 수행__한다.

## 4-1. 최적화
- 머신러닝은 __매개변수를 튜닝하기 위해__ 다양한 알고리즘의 __최적화 기법을 이용__한다. 따라서 최적화의 기초를 알아두는 것이 좋다.
- 기울기 하강법을 알아보기 전에 볼록과 비볼록 함수에 관해 먼저 알아보자. 함수 내의 어떤 두 점을 연결하는 직선도 함수보다 위에 있거나 함수에 있으면 볼록 함수라고 하고, 그렇지 않은 모든 경우를 비볼록 함수라고 한다. 함수가 볼록인지 아닌지를 아는 것은 매우 중요하다. 볼록 함수의 경우 국소 최적값이 곧 전역 최적값이 되지만 비볼록 함수에서는 국소 최적값이 전역 최적값이라는 것이 보장되지 않는다.
- 문제가 어렵게 느껴지는가? 방향을 조금 바꿔 __서로 다른 임의의 위치에서 탐색을 시작하는 프로세스를 생각해보자. 이렇게 하면 대개 전역 최적값에 수렴하기 마련__이다.

### 4-1-2. 기울기 하강법 (Gradient Descent)
- 이 기법은 __목적함수를 최소화__하는데, 목적 함수는 모델의 매개변수를 사용해 매개변수화돼 있다. 최소화 과정은 목적 함수의 경사와 반대 방향으로 __매개변숫값을 지속적으로 갱신하면서 이뤄진다.__ __학습률 설정에 따라 최소화에 이르기까지 필요한 단계가 달라진다.__

### 4-1-3. 완전 일괄 처리 기울기 하강법 (Full Batchh Gradient Descent)
- 이 방법은 __매 반복 시 전체(모든 훈련)관측 값을 고려__한다.
- 이 기법은 __엄청난 메모리 자원을 차지하고 속도 또한 느리다__. __사실 가중값 갱신을 위해 반드시 모든 관측값을 고려할 필요는 없다.__
- 그럼에도 불구하고 이 방법은 엄청난 계산양을 대가로 잡음이 적은 매개변수를 찾을 수 있는 최선의 방법을 제공한다.

### 4-1-4.  확률적 기울기 하강법
- 이 방법은 __각 반복 때마다 하나의 관측 값만 선택해 가중값을 갱신__한다.
- 이 방법은 가중값을 횡단하는 __가장 빠른 방법__이다. 그러나 __수렴 과정에 많음 잡음이 끼어든다.__

### 4-1-5.  미니 일괄 처리 기울기 하강법
- 이 방법은 엄청난 계산양과 빠른 가중값 갱신을 __절충한 방법__이다. __각 반복 시 임의의 30개 관측값을 선택해 경사를 계산하고 모델의 가중값을 갱신__한다.
- 여기서 특히 30여 개를 선택한 이유는 통계학 기초에 따라 표본이 모집단을 근사하게 되는 관측값의 수를 선택했기 때문이다. 그러나 배치 크기를 40이나 50으로 해도 당연히 무방하다. 전문가적 입장에서 최적의 모델을 찾기 위해 배치 크기를 바꿔가며 결과를 검증해보는 것이 좋다.

## 4-2. 선형회귀 vs 기울기 하강법